<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/04_mcp/05_mcp_stdio_with_inspector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q mcp uvicorn nest_asyncio

In [2]:
import nest_asyncio
import sys
import os

In [3]:
# 1. Apply the patch allows nested async loops in Colab
nest_asyncio.apply()

In [4]:
%%writefile server.py
import logging
import sys

# 1. Silence all logs (Fixes the "INFO Processing..." noise)
logging.basicConfig(level=logging.ERROR)

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("MathServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    return a * b

if __name__ == "__main__":
    mcp.run()

Writing server.py


In [5]:
%%writefile client.py
import asyncio
import sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def run():
    # 1. Launch server in UNBUFFERED mode ("-u")
    server_params = StdioServerParameters(
        command=sys.executable,
        args=["-u", "server.py"], # <--- THE FIX
    )

    print("🔌 Client connecting...")

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Test Add
            print(">> Requesting: add(100, 50)")
            result = await session.call_tool("add", arguments={"a": 100, "b": 50})
            print(f"✅ Result: {result.content[0].text}")

            # Test Multiply
            print(">> Requesting: multiply(5, 5)")
            result = await session.call_tool("multiply", arguments={"a": 5, "b": 5})
            print(f"✅ Result: {result.content[0].text}")

if __name__ == "__main__":
    asyncio.run(run())

Writing client.py


In [13]:
!python client.py

🔌 Client connecting...
>> Requesting: add(100, 50)
✅ Result: 150
>> Requesting: multiply(5, 5)
✅ Result: 25


In [7]:
# Cell 1 – Install Node.js 22 (one-liner, works every time)
!curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash - && sudo apt-get install -y nodejs
!node -v   # should show v22.x

2025-12-04 05:18:10 - Installing pre-requisites
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,563 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.c

In [8]:
# Cell 2 – Write the tiny config so the inspector knows where your Toolbox server lives
%%writefile mcp-inspector-config.json
{
  "mcpServers": {
    "toolbox-local": {
      "type": "streamable-http",
      "url": "http://localhost:5000/mcp"
    }
  },
  "defaultServer": "toolbox-local"
}

Writing mcp-inspector-config.json


In [10]:
# 3. Kill any old inspector (clean start) + launch it on the known port 6274
!pkill -f "@modelcontextprotocol/inspector" 2>/dev/null || true
!npx -y @modelcontextprotocol/inspector --config inspector-config.json --port 6274 > /dev/null 2>&1 &

import time
time.sleep(9)  # plenty of time even on slow Colab VMs
print("Inspector is warming up...")

^C
Inspector is warming up...


In [15]:
# 4. Magic line → public URL with Colab’s built-in secure tunnel (no token ever)
from google.colab.output import eval_js
print("MCP Inspector is ready!")
print("Click →", eval_js("google.colab.kernel.proxyPort(6274)"))

MCP Inspector is ready!
Click → https://6274-m-s-324ttf3b68vy0-b.us-east1-0.prod.colab.dev


In [16]:
!curl -s -o /dev/null -w "%{http_code}" http://localhost:6274 || echo "Not ready yet—rerun cell 2 and wait longer"

000Not ready yet—rerun cell 2 and wait longer


In [17]:
from google.colab import output
output.serve_kernel_port_as_window(6274)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>